# Collision tutorial

we'll demonstrate self collision on a sawyer robot

You can see that arm of the sawyer robot collide with milk object.

## Setup
If you want to check robot's collision, install python-fcl 

And then, import CollisionManager

In [1]:
import numpy as np
import yaml
import trimesh
import os

from pykin.robots.single_arm import SingleArm
from pykin.kinematics.transform import Transform
from pykin.collision.collision_manager import CollisionManager
from pykin.utils.collision_utils import apply_robot_to_scene
from pykin.utils.kin_utils import ShellColors as sc

In [2]:
pykin_path = os.path.dirname(os.getcwd())
file_path = '../asset/urdf/sawyer/sawyer.urdf'
mesh_path = pykin_path+"/asset/urdf/sawyer/"
config_path = '../asset/config/sawyer_init_params.yaml'

with open(config_path) as f:
    controller_config = yaml.safe_load(f)
init_qpos = controller_config["init_qpos"]

robot = SingleArm(file_path, Transform(rot=[0.0, 0.0, 0.0], pos=[0, 0, -0.5]))
fk = robot.forward_kin(np.array(np.concatenate((np.zeros(1), init_qpos))))

scene = trimesh.Scene()
scene = apply_robot_to_scene(mesh_path=mesh_path, scene=scene, robot=robot, fk=fk)
scene.set_camera(np.array([np.pi/6, 0, np.pi/12]))

<trimesh.scene.Camera> FOV: [60. 45.] Resolution: [1024  512]

## Use CollisionManager

Create robot's collision manager using CollisionManager

And check self collision

In [3]:
c_manager = CollisionManager(mesh_path)
c_manager.setup_robot_collision(robot, fk)

# Check self Collision 
result, name, data = c_manager.in_collision_internal(return_names=True, return_data=True)
print(result, name)

False No object collided..


Create object's collision manager using CollisionManager

In [4]:
milk_path = pykin_path+"/asset/objects/meshes/milk.stl"
milk_mesh = trimesh.load_mesh(milk_path)
o_manager = CollisionManager(milk_path)
o_manager.add_object("milk1", gtype="mesh", gparam=milk_mesh, transform=Transform(pos=[0.0, 0, 0.1]).h_mat)
o_manager.add_object("milk2", gtype="mesh", gparam=milk_mesh, transform=Transform(pos=[0.1, 0, 0.1]).h_mat)
scene.add_geometry(milk_mesh, node_name="milk1", transform=Transform(pos=[0.0, 0, 0.1]).h_mat)
scene.add_geometry(milk_mesh, node_name="milk2", transform=Transform(pos=[0.1, 0, 0.1]).h_mat)

'milk2'

Check collision between robot and objects

In [5]:
result, name, data = o_manager.in_collision_other(c_manager, return_names=True, return_data=True)
print(f"{sc.FAIL}Collision occurs between {list(name)[0][0]} and {list(name)[0][1]}!!{sc.ENDC}")

Collision occurs between milk1 and sawyer_head!!


Check collision and distance between robot and objects

In [6]:
result = o_manager.get_distances_other(c_manager)
for (object, robot), dis in result.items():
    if dis <= 0.0:
        print(f"{sc.FAIL}Collision occurs!!{sc.ENDC} ", end="")
    print(f"object: {object}, link: {robot}, distance: {dis}")

object: milk1, link: sawyer_pedestal, distance: 0.530347
object: milk1, link: sawyer_right_arm_base_link, distance: 0.452704
object: milk1, link: sawyer_link_0, distance: 0.141455
Collision occurs!! object: milk1, link: sawyer_head, distance: 0.0
object: milk1, link: sawyer_link_1, distance: 0.158688
object: milk1, link: sawyer_link_2, distance: 0.166534
object: milk1, link: sawyer_link_3, distance: 0.160757
object: milk1, link: sawyer_link_4, distance: 0.244461
object: milk1, link: sawyer_link_5, distance: 0.424437
object: milk1, link: sawyer_link_6, distance: 0.482171
object: milk2, link: sawyer_pedestal, distance: 0.528197
object: milk2, link: sawyer_right_arm_base_link, distance: 0.452333
object: milk2, link: sawyer_link_0, distance: 0.146183
object: milk2, link: sawyer_head, distance: 0.036907
object: milk2, link: sawyer_link_1, distance: 0.144447
object: milk2, link: sawyer_link_2, distance: 0.126585
object: milk2, link: sawyer_link_3, distance: 0.063889
object: milk2, link: sawy

In [7]:
scene.show()

/Users/jindaejong/opt/anaconda3/envs/lets-do-mujoco/lib/python3.9/site-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
